In [2]:
%%bash
pip install tsfresh
pip install ipywidgets
jupyter nbextension enable --py --sys-prefix widgetsnbextension

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [3]:
%%bash
jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [131]:
import os
import inspect
import sys
import pandas as pd
import charts
import numpy as np
import pylab as pl
from scipy import spatial
from scipy.spatial import distance
from sklearn.cluster import MeanShift
import time
import math
from opengrid.library import houseprint

In [132]:
hp = houseprint.Houseprint()

Opening connection to Houseprint sheet
Opening spreadsheets
Parsing spreadsheets
34 Sites created
34 Devices created
97 sensors created
Houseprint parsing complete


In [133]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

In [134]:
gas = hp.find_sensor('212ce724e124fbde0fb649396375d099')
#gas = hp.find_sensor('a6028457292f1866b468b63adcc82ee3') 2
#gas = hp.find_sensor('a6028457292f1866b468b63adcc82ee3')
print gas


    Fluksosensor
    Key: 212ce724e124fbde0fb649396375d099
    Type: gas
    


In [135]:
head = pd.Timestamp('20151110')
tail = pd.Timestamp('20151118')
originalGraph = gas.get_data(head=head, tail=tail, diff=True, resample='min', unit='kW')
originalGraphDf=pd.DataFrame(originalGraph)
originalGraphDfDropNa = originalGraphDf.dropna()

Using tmpo database from /data/.tmpo/tmpo.sqlite3


<h3> Different compare methods </h3>

In [136]:
def diffArrays(array1,array2):
    array=[]
    if array1.index.size==array2.index.size:
        for i in range(0,array1.index.size):
            array.append(array2.iloc[i]-array1.iloc[i])
    df=pd.DataFrame(array)
    return df

<h4> Manhattan Distance: the method we will use </h4>

In [137]:
def euclideanDistance(s1,s2):
    #Verwacht een numpy array
    dist = np.linalg.norm(s1-s2)
    return dist

In [138]:
def manhattanDistance(s1,s2):
    difference=diffArrays(s1,s2)
    absDifference=abs(difference)
    sumOfValues=np.sum(absDifference)
    maxValues=np.maximum(s1,s2)
    maxValues=np.max(maxValues)
    manhattan=sumOfValues/maxValues
    return manhattan

In [139]:
def manhattanDistance2(s1,s2):
    dist = distance.cityblock(s1,s2)
    return dist

In [140]:
def paa_transform(timeSeries, n_pieces):
    splitted = np.array_split(timeSeries, n_pieces) ## along columns as we want
    return np.asarray(map(lambda xs: xs.mean(axis = 0), splitted))

In [141]:
def znormalization(ts):
    #Pakt mean, std van kolom.
    meanCalc = ts.mean(axis = 0)
    stdCalc = ts.std(axis = 0)
    return (ts - meanCalc) / stdCalc

In [142]:
def sax_transform(ts, n_pieces, alphabet):
    """
    ts: columns of which are time serieses represented by np.array
    n_pieces: number of segments in paa transformation
    alphabet: the letters to be translated to, e.g. "abcd", "ab"
    return np.array of ts's sax transformation
    Steps:
    1. znormalize
    2. paa
    3. find norm distribution breakpoints by scipy.stats
    4. convert paa transformation into strings
    """
    from scipy.stats import norm
    alphabet_sz = len(alphabet)
    thrholds = norm.ppf(np.linspace(1./alphabet_sz, 
                                    1-1./alphabet_sz, 
                                    alphabet_sz-1))
    def translate(ts_values):
        return np.asarray([(alphabet[0] if ts_value < thrholds[0]
                else (alphabet[-1] if ts_value > thrholds[-1]
                      else alphabet[np.where(thrholds <= ts_value)[0][-1]+1]))
                           for ts_value in ts_values])
    paa_ts = paa_transform(znormalization(ts), n_pieces)
    return np.apply_along_axis(translate, 0, paa_ts)

In [143]:
def makeArray(array1):
    arrayValues=[]    
    for i in range(0, array1.size-1):
        if(i<array1.size):
             arrayValues.append(ord(array1.iloc[i])-97)
    df=pd.DataFrame(arrayValues)
    return df

In [144]:

def makeDiffArray(array): #Excepts a letter array
    arrayValues=[]    
    for i in range(0, array.size-1):
        if(i<array.size):
             arrayValues.append(ord(array.iloc[i+1])-ord(array.iloc[i]))
    df=pd.DataFrame(arrayValues)
    return df          


In [145]:
def differenceNumbers(array): #Expects array of numbers @
    arrayValues=[]
    for i in range(0, array.index.size):
        arrayValues.append(np.diff(array.iloc[i], axis=-1))
    df=pd.DataFrame(arrayValues)
    return df

In [146]:
def diffArrays(array1,array2):
    array=[]
    if array1.index.size==array2.index.size:
        for i in range(0,array1.index.size):
            array.append(array2.iloc[i]-array1.iloc[i])
    df=pd.DataFrame(array)
    return df

In [147]:
zScores=znormalization(originalGraph)

In [148]:
 def looping_through_list(df_with_values, array_to_compare):
    start=time.time()
    #Create list where values on which the different methods will act will be saved temporarily
    #This list will be cleared every time
    values_list=[]
    #Create list with results of the methods
    result_list=[]
    compare_to_array=np.array(array_to_compare)
    i=0 #Loops through list arrayToCompare.size times
    j=0 #Amount of values considered, and calculated.
    array_with_values=np.array(df_with_values)
    #start and stop timestamp
    starting_time_stamp=df_with_values.index[0]
    stop_time_stamp=df_with_values.index[array_to_compare.size-1]
    
    #Threshold
    threshold_value=array_to_compare.size/10
    #LOOPING THROUGH LIST
    while i < array_to_compare.size+j: #i is dependent on j, i updates as j updates.
        #Save x amount of values where x is the size of the array to compare.
        values_list.append(([array_with_values[i][0]]))
        i=i+1
        if(i==(array_to_compare.size+j)) and (i<(df_with_values.size)):
            #j = the minute at this moment. i = starting from the current minute, adding the size of the array to compare.
            #Stops running when the limit of values to be considered is reached, being the size the original array.
            if(j==0) or (manhattanDistance2(compare_to_array,values_list)!=result_list[len(result_list)-1][0]):
                #Create list with unique values. Save them in "resultList". Euclidean score, startTimestamp, stopTimestamp
                result_list.append([manhattanDistance2(compare_to_array,values_list), starting_time_stamp,stop_time_stamp]) 
            j=j+1
            starting_time_stamp=df_with_values.index[j] #startingTimeStamp: current minute considered
            stop_time_stamp=df_with_values.index[i] #stopTimeStamp: current minute considered + size of array to compare.
            i=j
            values_list=[]
    print time.time()-start
    return result_list

In [149]:
 def local_minima_ts_correction(result_list):
    start=time.time()
    loop_index=0 #Loops throu"gh the resultslist, as long as its size
    current_saves=0 #Current amount of variables saved
    some_list=[]
    df_result=pd.DataFrame(some_list)
    ts=(list_view[0][2]-list_view[0][1])/10
    threshold_value=(ts / np.timedelta64(1, 'm')).astype(int)
    #SAVING ONLY RELEVANT VALUES       
    while loop_index < len(result_list):
        if(loop_index==0):
            [value,start_time_stamp,stop_time_stamp] = [result_list[loop_index][0], result_list[loop_index][1], result_list[loop_index][2]]
            df_result=df_result.append(pd.DataFrame([[value, start_time_stamp,stop_time_stamp]], index=[current_saves], columns=['Manhattan','startTimeStamp','stopTimeStamp']))
            current_saves=current_saves+1
        if(result_list[loop_index-1][0] < result_list[loop_index-2][0]) and (result_list[loop_index-1][0] < result_list[loop_index][0]) and result_list[loop_index-1][0] < df_result.max()['Manhattan']:
            #print "Value", resultList[loopIndex-1][0],",Index:",loopIndex-1,"has a lower value than value left and right to it.(",resultList[loopIndex-2][0],",",resultList[loopIndex][0],") nIt will now replace", dfResult['Euclidean'].max(),"in the dataset."
            [value,start_time_stamp,stop_time_stamp] = [result_list[loop_index-1][0], result_list[loop_index-1][1], result_list[loop_index-1][2]]
            list_with_values=[]
            i=0
            max_value=0
            #!!!Timestamps are important. If timestamp is within the range of another timestamp already present,
            #they will overwrite eachother instead of adding a new unique value
            while i < df_result.index.size:
                if(start_time_stamp >= df_result['startTimeStamp'][i]) and (start_time_stamp <= df_result['stopTimeStamp'][i]-pd.Timedelta(minutes=threshold_value)):
                    #INSIDE BOUNDARIES
                    if(df_result.loc[i][0] > max_value) and (df_result.loc[i][0]!=0): #Store the maximum value, of the range between start and stoptimestamp.
                        max_value=df_result.loc[i][0]
                        ts=df_result.loc[i][1]
                    elif(df_result.loc[i][0]==0): #PERFECT MATCH
                        max_value=1 #Not 0 because otherwise we assume it is outside of the boundaries
                i=i+1
            if(max_value==0): 
                #OUTSIDE BOUNDARIES
                    df_result=df_result.append(pd.DataFrame([[value, start_time_stamp,stop_time_stamp]], index=[current_saves], columns=['Manhattan','startTimeStamp','stopTimeStamp']))
                    current_saves=current_saves+1
                #INSIDE BOUNDARIES - 
            elif(value < max_value): #Check if the current value is smaller than the Maximum value encountered. Replace if it is.
                    df_result.loc[df_result['Manhattan']== max_value] = [value, start_time_stamp, stop_time_stamp]
        loop_index=loop_index+1
    df_result=df_result.sort_values(['Manhattan'])
    print time.time()-start
    return df_result
        

In [150]:
def calculate_threshold_value(df_result, nr_of_best_values, factor):
    list_of_best_values=[]
    i = 0
    for i in range(0, nr_of_best_values):
        list_of_best_values.append(df_result.iloc[i])
    list_of_best_values=pd.DataFrame(list_of_best_values)
    mean=list_of_best_values.mean()
    threshold=mean*factor
    print threshold
    return df_result.loc[df_result['Manhattan']<=threshold[0]]
    

In [151]:
def create_graph_from(df):
    listWithGraphs=[]
    listWithGraphs=pd.DataFrame(listWithGraphs)
    for i in range (0, df.index.size):
        listWithGraphs=listWithGraphs.append(originalGraphDfDropNa.ix[pd.Timestamp(df['startTimeStamp'][i]):pd.Timestamp(df['stopTimeStamp'][i])])
    return charts.plot(listWithGraphs, stock=True, show='inline')

In [152]:
head = pd.Timestamp('20151110')
tail = pd.Timestamp('20151128')
originalGraph = gas.get_data(head=head, tail=tail, diff=True, resample='min', unit='kW')
originalGraphDf=pd.DataFrame(originalGraph)
originalGraphDfDropNa = originalGraphDf.dropna()

In [153]:
dfCompareOriginal=originalGraphDf.ix[pd.Timestamp('2015-11-13 16:25:00+00:00'):pd.Timestamp('2015-11-13 17:16:00+00:00')]

In [104]:
list_view=looping_through_list(originalGraphDfDropNa, dfCompareOriginal)
df_result = local_minima_ts_correction(list_view)


75.8810019493
34.3666110039
Manhattan    121.656511
dtype: float64


In [119]:
df_threshold=calculate_threshold_value(df_result, 20,1.5)

Manhattan    121.656511
dtype: float64


In [120]:
df_threshold=df_threshold.sort_values(['startTimeStamp'])
create_graph_from(df_threshold)

<h2> Pickle use </h2>

In [116]:
import pickle

In [127]:
pickle_out = open("df_threshold.pickle", "wb")
pickle.dump(df_threshold, pickle_out)

In [128]:
pickle_in = open("df_threshold.pickle", "rb")
example = pickle.load(pickle_in)

In [130]:
example

,Manhattan,startTimeStamp,stopTimeStamp
14,64.541744,2015-11-12 09:57:00+00:00,2015-11-12 10:48:00+00:00
16,108.904708,2015-11-12 19:14:00+00:00,2015-11-12 20:05:00+00:00
21,0.000000,2015-11-13 16:25:00+00:00,2015-11-13 17:16:00+00:00
26,51.040555,2015-11-14 06:04:00+00:00,2015-11-14 06:55:00+00:00
27,55.597896,2015-11-14 14:13:00+00:00,2015-11-14 15:04:00+00:00
32,103.841869,2015-11-15 05:47:00+00:00,2015-11-15 06:38:00+00:00
33,93.254822,2015-11-15 06:49:00+00:00,2015-11-15 07:40:00+00:00
35,70.112119,2015-11-15 16:07:00+00:00,2015-11-15 16:58:00+00:00
41,71.200876,2015-11-16 08:28:00+00:00,2015-11-16 09:19:00+00:00
50,52.612096,2015-11-17 17:11:00+00:00,2015-11-17 18:02:00+00:00
